## Dataset Cleaning

In [ ]:
# %pip install fuzzywuzzy
# !python -m pip install --upgrade pip
# !pip install pycountry
# %pip install python-Levenshtein
# %pip install pandas numpy matplotlib seaborn scikit-learn jupyter

# errors='coerce' in pd.to_numeric() means:
# “Try to convert everything in this column into a number.
# If it fails (e.g. the value is text or invalid), don’t crash — just replace it with NaN.”

In [ ]:
# Activate kernal:
# .\venv_fyp\Scripts\activate
import common_utils, customer_cleaning, order_cleaning
# Step 1: Import libraries
import pandas as pd
import numpy as np
import os
import pycountry
import re
import requests
import time
from datetime import datetime, date
from fuzzywuzzy import process, fuzz

In [ ]:
# choose file

# order_file_path = "C:/Users/user/OneDrive/Desktop/Onedrive_YuyanDipsy/OneDrive/UM Y4S1/WIA3002 FYP 1 & 2/FYP2/Data/Soapan Santun - Use/2021 - 2025 Order.csv"
# customer_file_path = "C:/Users/user/OneDrive/Desktop/Onedrive_YuyanDipsy/OneDrive/UM Y4S1/WIA3002 FYP 1 & 2/FYP2/Data/Soapan Santun - Use/2021 - 2025 Customer - Copy.csv"

file_path = "C:/Users/user/OneDrive/Desktop/Onedrive_YuyanDipsy/OneDrive/UM Y4S1/WIA3002 FYP 1 & 2/FYP2/Data/Soapan Santun - Use/2021 - 2025 Customer - Copy.csv"
original_file_name = "2021 - 2025 Customer - Copy.csv"
dataset_type = "customer"

# Read CSV
df = pd.read_csv(file_path)

# Generate output paths
temp_dir = os.path.dirname(file_path)
base_name, ext = os.path.splitext(original_file_name)

cleaned_path = os.path.join(temp_dir, f"{base_name}_cleaned{ext}")
report_path  = os.path.join(temp_dir, f"{base_name}_report.json")

# Run cleaning pipeline
if dataset_type == "customer":
    cleaned_df, report = customer_cleaning.clean_customer_dataset(df, cleaned_path)
else:
    cleaned_df, report = order_cleaning.clean_order_dataset(df, cleaned_path)

# Save report
with open(report_path, "w", encoding="utf-8") as f:
    json.dump(report, f, indent=4, default=str)

cleaned_df.head(), report


### Generic Function

In [ ]:
# # ============================================ GENERIC FUNCTIONS ============================================ #

# def normalize_columns_name(df):
#     """Normalize column names: lowercase, strip spaces"""
#     print("[LOG - STAGE 0] Running normalize_columns_name...")
#     df.columns = df.columns.str.strip().str.lower()
#     print(f"[LOG - STAGE 0] Columns after normalization: {list(df.columns)}")
#     return df

# def check_mandatory_columns(df, dataset_type, mandatory_columns, threshold=0.9):
#     """
#     Generic function to check mandatory columns for both Customer and Order datasets.
#     - dataset_type: 'customer' or 'order'
#     - mandatory_columns: list of required columns for that dataset
#     - threshold: minimum acceptable fill ratio (default 0.8)
#     """
#     print(f"[LOG - STAGE 1] Running check_mandatory_columns for {dataset_type} dataset...")

#     missing_report = []
#     warning_columns = []

#     # Step 1: Check each mandatory column
#     for col in mandatory_columns:
#         if col in df.columns:
#             fill_ratio = df[col].notna().mean()
#             print(f"[LOG - STAGE 1] Mandatory column '{col}' fill ratio: {fill_ratio:.2f}")
#             missing_percent = (1 - fill_ratio) * 100
#             missing_report.append(f"{col}: {missing_percent:.1f}% missing")

#             if fill_ratio < threshold:
#                 warning_columns.append(col)
#         else:
#             print(f"[LOG - STAGE 1] Mandatory column '{col}' not found")
#             missing_report.append(f"{col}: column not found (100% missing)")
#             warning_columns.append(col)

#     # Step 2: Generate message
#     if warning_columns:
#         warning_str = ", ".join(warning_columns)
#         warning_details = [report for report in missing_report if any(col in report for col in warning_columns)]
        
#         # Check if location fields (city/state) are in warning
#         location_warning = any(col in ['city', 'state'] for col in warning_columns)
        
#         message = (
#             f"⚠️ WARNING - Critical Data Gaps Detected:\n\n"
#             f"The following MANDATORY field(s) have excessive missing values (>10%): {warning_str}\n\n"
#         )
#         if warning_details:
#             message += "Missing Data Details:\n" + "\n".join(f"  • {d}" for d in warning_details) + "\n\n"
        
#         message += (
#             f"⚠️ Impact: Missing mandatory data can significantly reduce the accuracy of your {dataset_type} segmentation results.\n\n"
#             f"How Missing Values Will Be Handled:\n"
#         )
        
#         # Add specific handling explanation based on dataset type and fields
#         if dataset_type == "customer":
#             message += (
#                 f"  • CustomerID: Rows without ID will be REMOVED (cannot be predicted)\n"
#             )
#             if location_warning:
#                 message += (
#                     f"  • Location (City/State): Missing values will be filled using:\n"
#                     f"      1. Geocoding API (if city known, find correct state)\n"
#                     f"      2. Statistical mode (most frequent city/state in your data)\n"
#                     f"      3. If both missing: use most common city-state pair\n"
#                     f"    ⚠️ Note: This creates estimated data which may not reflect true customer locations\n"
#                 )
#             message += (
#                 f"  • Demographics (Age/Gender): Missing values will be kept as 'Unknown' for segmentation\n"
#             )
#         elif dataset_type == "order":
#             message += (
#                 f"  • OrderID/CustomerID: Rows without IDs will be REMOVED\n"
#                 f"  • Financial data (Item Price/Total Spend): Missing values will be calculated or rows removed\n"
#                 f"  • Other fields: Filled with statistical defaults or 'Unknown'\n"
#             )
        
#         message += (
#             f"\n🔴 STRONGLY RECOMMENDED:\n"
#             f"{'=' * 70}\n"
#             f"Please REVIEW and REUPLOAD your source data with complete information!\n"
#         )
        
#         if location_warning:
#             message += (
#                 f"\n⚠️ LOCATION DATA IS CRITICAL for accurate customer segmentation!\n"
#                 f"   Filling missing locations with statistical estimates may lead to:\n"
#                 f"   • Incorrect geographic segments\n"
#                 f"   • Misleading regional analysis\n"
#                 f"   • Poor targeting accuracy\n"
#                 f"\n   For best results, provide complete City and State information.\n"
#             )
        
#         message += f"{'=' * 70}\n"
        
#     else:
#         complete_details = "\n".join(f"  • {report}" for report in missing_report)
#         message = (
#             f"✅ All Mandatory Fields Validated:\n\n"
#             f"All required {dataset_type} columns have sufficient data (≥90% filled) and are ready for cleaning.\n\n"
#             f"Data Completeness:\n{complete_details}\n\n"
#             f"Note: We already clean missing values in these columns during the cleaning process."
#         )

#     return df, message

# def remove_duplicate_entries(df):
#     """Remove duplicate rows, keeping the first occurrence"""
#     print("[LOG - STAGE 2] Running remove_duplicate_entries...")
#     initial_len = len(df)
#     df = df.drop_duplicates(keep='first', ignore_index=True)
#     removed_dup = initial_len - len(df)
#     message = None
#     if removed_dup > 0:
#         message = (f"{removed_dup} duplicate records have found, we have removed it.")
#     print(f"[LOG - STAGE 2] Removed {initial_len - len(df)} duplicate rows.")
#     return df, message

# def standardize_customer_id(df):
#     """Standardize CustomerID format and keep null as NaN"""
#     print("[LOG - STAGE 4] Running standardize_customer_id...")

#     if 'customerid' in df.columns:
#         # Convert to string, strip spaces
#         df.loc[:, 'customerid'] = df['customerid'].astype(str).str.strip().str.upper()

#         # Convert empty string back to NaN
#         df.loc[df['customerid'] == '', 'customerid'] = np.nan

#         print("[LOG - STAGE 4] CustomerID column standardized (empty -> NaN)")
#     else:
#         print("[LOG - STAGE 4] CustomerID column not found, skipping")

#     return df
# # Might have special case of dirty data exist such as "****", "1234....", "annbwbciwbciowb"
# # not sure how to handle it (Currently will say bcs we focus on small business enterprise that have use digital system, so normally customerID will not have inconsistent format issue, even the inconsistant format exist, at the end this row will not be use as when we merge we cant found that customerID)


## Perform Data Cleaning Pipeline - CustomerDataset

In [ ]:
# # Step 2: Load dataset
# # Replace 'customer_dataset.csv' with your actual file name or path
# customer_file_path = "C:/Users/user/OneDrive/Desktop/Onedrive_YuyanDipsy/OneDrive/UM Y4S1/WIA3002 FYP 1 & 2/FYP2/Data/Soapan Santun - Use/2021 - 2025 Customer - Copy.csv"
# # customer_file_path = "C:/Users/user/OneDrive/Desktop/Onedrive_YuyanDipsy/OneDrive/UM Y4S1/WIA3002 FYP 1 & 2/FYP2/Data/Test Data/customer_dataset_balanced_5033_rows.csv"
# # customer_file_path = "C:/Users/user/OneDrive/Desktop/Onedrive_YuyanDipsy/OneDrive/UM Y4S1/WIA3002 FYP 1 & 2/FYP2/Data/Test Data/customer_dataset_light_7719_rows.csv"
# # customer_file_path = "C:/Users/user/OneDrive/Desktop/Onedrive_YuyanDipsy/OneDrive/UM Y4S1/WIA3002 FYP 1 & 2/FYP2/Data/Test Data/customer_dataset_stress_6047_rows.csv"

# original_customer_dataset_name = "2021 - 2025 Customer - Copy.csv"

# # Read dataset
# customer_df = pd.read_csv(customer_file_path)

# # Show first few rows (original raw data)
# customer_df.head()

In [ ]:
# # Step 2: Load dataset
# # Replace 'customer_dataset.csv' with your actual file name or path
# order_file_path = "C:/Users/user/OneDrive/Desktop/Onedrive_YuyanDipsy/OneDrive/UM Y4S1/WIA3002 FYP 1 & 2/FYP2/Data/Soapan Santun - Use/2021 - 2025 Order.csv"
# # customer_file_path = "C:/Users/user/OneDrive/Desktop/Onedrive_YuyanDipsy/OneDrive/UM Y4S1/WIA3002 FYP 1 & 2/FYP2/Data/Test Data/customer_dataset_balanced_5033_rows.csv"
# # customer_file_path = "C:/Users/user/OneDrive/Desktop/Onedrive_YuyanDipsy/OneDrive/UM Y4S1/WIA3002 FYP 1 & 2/FYP2/Data/Test Data/customer_dataset_light_7719_rows.csv"
# # customer_file_path = "C:/Users/user/OneDrive/Desktop/Onedrive_YuyanDipsy/OneDrive/UM Y4S1/WIA3002 FYP 1 & 2/FYP2/Data/Test Data/customer_dataset_stress_6047_rows.csv"

# original_order_dataset_name = "2021 - 2025 Order.csv"

# # Read dataset
# order_df = pd.read_csv(order_file_path)

# # Show first few rows (original raw data)
# order_df.head()

In [ ]:
# # Step 1: Import libraries
# # from common_utils import normalize_columns_name, check_mandatory_columns, remove_duplicate_entries, standardize_customer_id; 
# import pandas as pd
# import numpy as np
# import os
# import pycountry
# import re
# import requests
# import time
# from datetime import datetime, date
# from fuzzywuzzy import process, fuzz

# # ============================================= (CUSTOMER DATASET) STAGE 0: NORMALIZE COLUMN NAMES =============================================
# # From Generic function: normalize_columns_name

# # ============================================= (CUSTOMER DATASET) STAGE 1: SCHEMA & COLUMN VALIDATION =============================================
# # Optional columns & Mandatory columns(FROM GENERIC FUNCTION - check_mandatory_columns)
# def customer_check_optional_columns(df, threshold=0.9):
#     """
#     Check optional columns for fill percentage and drop columns that are mostly empty.
#     Returns the modified DataFrame and a friendly message.
#     """
#     print("[LOG - STAGE 1] Running customer_check_optional_columns...")
#     optional_columns = ["date of birth", "gender"]
#     dropped_columns = []
#     missing_report = []

#     for col in optional_columns:
#         if col in df.columns:
#             fill_ratio = df[col].notna().mean()
#             missing_percent = (1 - fill_ratio) * 100
#             missing_report.append(f"{col}: {missing_percent:.1f}% missing")
#             print(f"[LOG - STAGE 1] Optional column '{col}' fill ratio: {fill_ratio:.2f}")
#             if fill_ratio < threshold:
#                 dropped_columns.append(col)
#                 df.drop(columns=[col], inplace=True)  # Drop the column immediately
#                 # df[col].count(): This counts the number of non-missing (non-null/non-NaN) values in the current column (col).
#                 # len(df): This gives the total number of rows in the DataFrame.
#                 # fill_ratio: The division calculates the proportion of filled (non-missing) values in that column. A ratio of 1.0 means the column is entirely filled; a ratio of 0.1 means 90% of the values are missing.
#                 print(f"[LOG - STAGE 1] Dropped optional column '{col}' due to too many missing values")
#         else:
#             print(f"[LOG - STAGE 1] Optional column '{col}' not found")
#             missing_report.append(f"{col}: column not found (100% missing)")
#             dropped_columns.append(col)

#     # Generate user-friendly message
#     if dropped_columns:
#         dropped_str = ", ".join(dropped_columns)
#         dropped_details = [report for report in missing_report if any(col in report for col in dropped_columns)]
#         kept_details = [report for report in missing_report if not any(col in report for col in dropped_columns)]
        
#         message = (
#             f"Optional Columns Removed:\n\n"
#             f"The following column(s) were removed due to insufficient data: {dropped_str}\n"
#         )
#         if dropped_details:
#             message += "Details:\n" + "\n".join(f"  • {d}" for d in dropped_details) + "\n\n"
        
#         message += (
#             "Why? These columns had less than 90% complete data, which is too sparse for reliable analysis.\n\n"
#             "Good News: Segmentation will still work using:\n"
#             "  • Geographic data (City, State)\n"
#             "  • Behavioral data (orders, purchase patterns, spending)\n"
#         )
        
#         if kept_details:
#             message += "\n\n Columns Kept:\n" + "\n".join(f"  • {d}" for d in kept_details)
#     else:
#         message = (
#             "All Optional Columns Retained:\n\n"
#             "All optional columns (Date of Birth, Gender) have sufficient data (≥90% filled) and will be kept for analysis.\n\n"
#             "Data Completeness:\n" + "\n".join(f"  • {report}" for report in missing_report) + "\n\n"
#             "Note: We already clean missing values in these columns during the cleaning process."
#         )
    
#     return df, message

# # ============================================= (CUSTOMER DATASET) STAGE 2: REMOVE DUPLICATE ENTRY ROW =================================================
# # From Generic function: remove_duplicate_entries

# # ============================================= (CUSTOMER DATASET) STAGE 3: DEDUPLICATE =================================================
# def deduplicate_customers(df):
#     print("[LOG - STAGE 3] Running deduplicate_customers...")
#     if 'customerid' not in df.columns:
#         print("[LOG - STAGE 3] 'customerid' column missing, skipping deduplication")
#         return df

#     before_dup_id = len(df)
#     def resolve_conflict(series):
#         vals = series.dropna().unique()
#         if len(vals) == 0:
#             return pd.NA
#         elif len(vals) == 1:
#             return vals[0]
#         else:
#             return series.mode().iloc[0]

#     # Vectorized groupby instead of per-group loop
#     df = df.groupby('customerid', as_index=False).agg(resolve_conflict)
    
#     removed_dup_id = before_dup_id - len(df)
#     if removed_dup_id > 0:
#         message = (f"{removed_dup_id} duplicate CustomerIDs removed.")
#     else:
#         message = "No duplicate CustomerIDs found."
#     print("[LOG - STAGE 3] Deduplication complete (vectorized)")
#     return df, message

# # ============================================= (CUSTOMER DATASET) STAGE 4: STANDARDIZATION & NORMALIZATION =============================================
# # From Generic function: standardize_customer_id

# def standardize_dob(df):
#     """Standardize Date of Birth column and convert to YYYY-MM-DD"""
#     print("[LOG - STAGE 4] Running standardize_dob...")
#     # Rename only 'date of birth' to 'dob'
#     message = None
#     df = df.rename(columns={'date of birth': 'dob'})  
#     if 'dob' in df.columns:
#         print("[LOG - STAGE 4] DOB column found, parsing dates...")
#         def parse_date(x):
#             if pd.isnull(x):
#                 return pd.NaT
#             for format in ("%d/%m/%Y", "%m-%d-%y", "%Y-%m-%d", "%d-%b-%Y", "%d-%m-%Y", "%d %B %Y", "%B %d, %Y", "%b %d %Y"):    
#                 try:
#                     return datetime.strptime(str(x), format).date() # Final format: YYYY-MM-DD | 2025-10-15
#                 except Exception:
#                     continue
#             return pd.NaT  # If no valid format found
#         df['dob'] = df['dob'].apply(parse_date)
#         df['dob'] = pd.to_datetime(df['dob'])
#         # Keep NaT as is - don't fill with "Unknown" string (causes issues with .year/.month/.day)
#         print("[LOG - STAGE 4] DOB parsing complete. Invalid dates kept as NaT")
#         message = (
#             "Since your dataset includes a Date of Birth information, we derived two useful fields — "
#             "'age' and 'age_group' — for segmentation purposes, and the original 'Date of Birth' column will be remove."
#         )
#     else:
#         print("[LOG - STAGE 4] DOB column not found, skipping")
#     return df, message

#     # %d/%m/%Y → 12/05/2000
#     # %m-%d-%y → 05-12-00
#     # %Y-%m-%d → 2000-05-12
#     # %d-%b-%Y → 12-May-2000
#     # %d-%m-%Y → 12-5-2000
#     # "%d %B %Y" → 15 October 1998
#     # "%B %d, %Y" → October 15, 1998
#     # "%b %d %Y" → Oct 15 1998

# # ===============================================================================

# def derive_age_features(df):
#     """Derive Age from DOB"""
#     print("[LOG - STAGE 4] Running derive_age_features...")
#     if 'dob' in df.columns:
#         today = date.today()
#         df['age'] = df['dob'].apply(
#             lambda x: today.year - x.year - ((today.month, today.day) < (x.month, x.day))
#             if pd.notnull(x) else None
#         )
#         df['age'] = pd.to_numeric(df['age'], errors='coerce')
#         # Replace NaN with "Unknown" and ensure no empty strings
#         df['age'] = df['age'].fillna("Unknown")
#         df['age'] = df['age'].replace('', 'Unknown')  # Handle empty strings
#         print("[LOG - STAGE 4] Age derived from DOB, null age marked as Unknown")
#     else:
#         # If DOB column doesn't exist, skip age creation
#         print("[LOG - STAGE 4] DOB column not found, skipping age creation")
#     return df
#     # Example: ((today.month, today.day) < (x.month, x.day))
#     # (10,15) < (12,1) → True (birthday in Dec is after Oct 15)
#     # (10,15) < (10,16) → True (birthday tomorrow)
#     # (10,15) < (5,20) → False (birthday already passed)

#     # This function calculates each person’s age from their date of birth (dob) by subtracting their birth year from the current year and adjusting if their birthday hasn’t occurred yet this year.

# # ===============================================================================

# def derive_age_group(df):
#     """Derive Age Group based on defined buckets"""
#     print("[LOG - STAGE 4] Running derive_age_group...")
#     if 'age' in df.columns:
#         def categorize_age(age):
#             if pd.isnull(age) or age == "Unknown" or age == '':
#                 return 'Unknown'
#             # Convert to numeric if string (safety check)
#             if isinstance(age, str):
#                 try:
#                     age = float(age)
#                 except:
#                     return 'Unknown'
#             if age < 18: return 'Below 18'
#             elif 18 <= age <= 24: return '18-24'
#             elif 25 <= age <= 34: return '25-34'
#             elif 35 <= age <= 44: return '35-44'
#             elif 45 <= age <= 54: return '45-54'
#             elif 55 <= age <= 64: return '55-64'
#             else: return 'Above 65'
#         df['age_group'] = df['age'].apply(categorize_age)
#         # Clean up any remaining empty strings or NaN
#         df['age_group'] = df['age_group'].fillna("Unknown")
#         df['age_group'] = df['age_group'].replace('', 'Unknown')
#         print("[LOG - STAGE 4] Age groups derived, null age_group marked as Unknown")
#     else:
#         print("[LOG - STAGE 4] Age column not found, skipping age_group creation")
#     return df

# # ===============================================================================

# def drop_dob_after_age_derived(df):
#     """Drop DOB column after deriving age and age_group"""
#     print("[LOG - STAGE 4] Running drop_dob_after_age_derived...")
#     if 'dob' in df.columns:
#         df = df.drop(columns=['dob'])
#         print("[LOG - STAGE 4] Dropped DOB column")
#     else:
#         print("[LOG - STAGE 4] DOB column not found, skipping")
#     return df

# # Column	    Value when original DOB is null
# # =============================================
# # dob	        "Unknown" 
# # age	        "Unknown" 
# # age_group	    "Unknown" 
# # =================================================================================

# def standardize_gender(df):
#     """Clean and standardize gender values"""
#     print("[LOG - STAGE 4] Running standardize_gender...")
#     if 'gender' in df.columns:
#         # Normalize to lowercase for mapping
#         gender_norm = df['gender'].astype(str).str.strip().str.lower()

#         # Map common variants to canonical labels
#         mapping = {
#             'm': 'Male', 'male': 'Male', 'man': 'Male', 'boy': 'Male',
#             'f': 'Female', 'female': 'Female', 'woman': 'Female', 'girl': 'Female'
#         }
#         gender_norm = gender_norm.replace(mapping)

#         # Anything not exactly 'Male' or 'Female' becomes Unknown (e.g., other/others/na/null/empty)
#         df['gender'] = gender_norm
#         df.loc[~df['gender'].isin(['Male', 'Female']), 'gender'] = 'Unknown'
#         print("[LOG - STAGE 4] Gender standardized (vectorized)")
#     else:
#         print("[LOG - STAGE 4] Gender column not found, skipping")
#     return df

# # ==================================================================================

# def standardize_location(df):
#     """Standardize City, and State fields"""
#     print("[LOG - STAGE 4] Running standardize_location...")
        
#     # Helper function: detect suspicious city names
#     def is_suspicious_city(name):
#         if not name or name.strip() == '':
#             return True
#         name = str(name).strip()
        
#         if len(name) < 2 or len(name) > 50: # Too short or too long
#             return True
        
#         if re.search(r'[^A-Za-z\s\'-]', name):  # Contains non-alphabetic or weird symbols   # letters, space, apostrophe, dash allowed
#             return True
        
#         if re.search(r'(.)\1{3,}', name):   # Repeated characters (e.g., "Ccciiiittty")
#             return True
#         return False

#     # --- City ---
#     if 'city' in df.columns:
#         df['city'] = df['city'].fillna('').astype(str).str.title().str.strip()
#         # Common city aliases (short forms, local spellings, etc.)
#         city_alias_map = {
#             "Kl": "Kuala Lumpur",
#             "PJ": "Petaling Jaya",
#         }
#         # Apply alias replacements first
#         df['city'] = df['city'].replace(city_alias_map)
        
#         suspicious_mask = df['city'].apply(lambda x: is_suspicious_city(x) or x.lower() in ['others', 'other'])
#         suspicious_count = suspicious_mask.sum()
#         df.loc[suspicious_mask, 'city'] = 'Unknown'
        
#         print(f"[LOG - STAGE 4] Standardized 'city'. Suspicious/unknown entries set to 'Unknown': {suspicious_count}")
#     else:
#         print("[LOG - STAGE 4] 'city' column not found, skipping city standardization")
    
#     # --- State ---
#     if 'state' in df.columns:
#         # malaysia_states = ["Johor", "Kedah", "Kelantan", "Melaka", "Negeri Sembilan","Pahang", "Perak", "Perlis", "Pulau Pinang", "Sabah", "Sarawak", "Selangor", "Terengganu", "Kuala Lumpur", "Labuan", "Putrajaya"]
#         malaysia_states = [sub.name for sub in pycountry.subdivisions if sub.country_code == 'MY']
#         alias_map = {
#             "Kuala Lumpur": "Wilayah Persekutuan Kuala Lumpur",
#             "Kl": "Wilayah Persekutuan Kuala Lumpur",
#             "Labuan": "Wilayah Persekutuan Labuan",
#             "Putrajaya": "Wilayah Persekutuan Putrajaya"
#         }

#         df['state'] = df['state'].fillna('').astype(str).str.title().str.strip()
#         # Match only unique states once
#         unique_states = df['state'].unique()
#         state_map = {}
#         for s in unique_states:
#             s_clean = s.strip().title()

#             if not s_clean or s_clean == 'Unknown':
#                 state_map[s] = 'Unknown'

#             elif s_clean in alias_map:   # Check alias first
#                 state_map[s] = alias_map[s_clean]

#             else:
#                 match, score = process.extractOne(s_clean, malaysia_states, scorer=fuzz.token_sort_ratio)
#                 state_map[s] = match if score >= 80 else 'Unknown'

#         # Apply mapping to the dataframe
#         df['state'] = df['state'].map(state_map)
#         print("[LOG - STAGE 4] State standardized (cached fuzzy matching)")
#     else:
#         print("[LOG - STAGE 4] 'state' column not found, skipping state standardization")

#     return df

# # ============================================= (CUSTOMER DATASET) STAGE 5: MISSING VALUE HANDLING =============================================
# # Only handle for customerid and location fields
# def handle_missing_values_customer(df):
#     print("[LOG - STAGE 5] Running handle_missing_values...")

#     API_KEY = "68f8ce9a38c3f632237334dyiedb96e"
#     GEOCODE_URL = "https://geocode.maps.co/search"
#     SLEEP_TIME = 1.2
#     cache = {}  # ⚡ moved outside loops
    
#     messages = []
#     stats = {
#         "customerid_removed": 0,
#         "case1_api_filled": 0,
#         "case1_fallback_filled": 0,
#         "case2_filled": 0,
#         "case3_filled": 0
#     }

#     # --- Drop rows without ID ---
#     if 'customerid' in df.columns:
#         before_drop = len(df)
#         df = df[df['customerid'].notna()].copy()
#         rows_removed = before_drop - len(df)
#         stats["customerid_removed"] = rows_removed
#         print(f"[LOG - STAGE 5] Dropped {rows_removed} rows without CustomerID")
#         if rows_removed > 0:
#             messages.append(f" Removed {rows_removed} record(s) with missing CustomerID as we cannot predict valid customer identifiers.")
#     else:
#         print("[LOG - STAGE 5] 'customerid' column missing, skipping drop")

#     # --- City & State handling ---
#     if {'city', 'state'}.issubset(df.columns):
#         print("\n[LOG - STAGE 5] Handling missing city/state values...")
#         malaysia_states = [sub.name for sub in pycountry.subdivisions if sub.country_code == 'MY']
#         cache = {}  # city -> validated state
#         SLEEP_TIME = 1.2
        
#         # Case 1: missing state but city known → fill via geocoding API
#         print("\n[LOG - STAGE 5] Case 1: Filling missing state where city is known...")
#         # Get rows needing state fill (city known, state unknown)
#         mask_case1 = (df['city'] != 'Unknown') & (df['state'] == 'Unknown')
#         cities_to_query = df.loc[mask_case1, 'city'].unique().tolist()

#         print(f"[LOG - STAGE 5] {len(cities_to_query)} unique cities need state lookup")
        
#         for city in cities_to_query:
#             if city not in cache:
#                 # Call API
#                 try:
#                     resp = requests.get(GEOCODE_URL, params={"q": f"{city}, Malaysia", "api_key": API_KEY}, timeout=10)
#                     if resp.status_code == 200:
#                         data = resp.json()
#                         if isinstance(data, list) and data:
#                             state_name = data[0].get("address", {}).get("state")
#                             if state_name and state_name in malaysia_states:
#                                 cache[city] = state_name
#                             else:
#                                 cache[city] = None
#                         else:
#                             cache[city] = None
#                     else:
#                         cache[city] = None
#                     time.sleep(SLEEP_TIME)
#                 except Exception as e:
#                     print(f"[WARN - STAGE 5] Failed to get state for city '{city}': {e}")
#                     cache[city] = None

#             # Fill values
#             fill_state = cache.get(city)
#             if fill_state:
#                 filled_count = ((df['city'] == city) & (df['state'] == 'Unknown')).sum()
#                 df.loc[(df['city'] == city) & (df['state'] == 'Unknown'), 'state'] = fill_state
#                 stats["case1_api_filled"] += filled_count
#                 print(f"[TRACE - STAGE 5] Filled {filled_count} → {city} → state='{fill_state}' (API valid)")
#             else:
#                 # Fallback: use mode state & mode city for that state
#                 # The API fails to find the city || The response doesn’t contain a valid "state" field || Or the returned "state" isn’t in the official Malaysia subdivision list.
#                 valid_states = df[df['state'] != 'Unknown']['state']
#                 mode_state = valid_states.mode()[0] if not valid_states.empty else 'Unknown'

#                 # Compute mode city per state
#                 mode_city_per_state = (
#                     df[df['city'] != 'Unknown'].groupby('state')['city']
#                     .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else 'Unknown')
#                     .to_dict()
#                 )
#                 mode_city = mode_city_per_state.get(mode_state, 'Unknown')

#                 mask_fill = (df['city'] == city) & (df['state'] == 'Unknown')
#                 filled_count = mask_fill.sum()
#                 df.loc[mask_fill, 'state'] = mode_state
#                 df.loc[mask_fill, 'city'] = mode_city
#                 stats["case1_fallback_filled"] += filled_count
#                 print(f"[TRACE - STAGE 5] Filled {filled_count} row(s) → city='{mode_city}', state='{mode_state}' (Fallback)")

#         if stats["case1_api_filled"] > 0 or stats["case1_fallback_filled"] > 0:
#             msg = "Location - Missing State (City Known):\n"
#             if stats["case1_api_filled"] > 0:
#                 msg += f"  • {stats['case1_api_filled']} record(s) automatically matched to correct state using map service\n"
#             if stats["case1_fallback_filled"] > 0:
#                 msg += f"  • {stats['case1_fallback_filled']} record(s) filled based on where most customers are located (city name couldn't be verified - may be misspelled or invalid)\n"
#             msg += "Why safe: We kept their original city and only filled in the missing state using reliable location data"
#             messages.append(msg)

#         # Case 2: missing city but state known → fill with mode city per state
#         print("\n[LOG - STAGE 5] Case 2: Filling missing city where state is known...")
#         mask_case2 = (df['city'] == 'Unknown') & (df['state'] != 'Unknown')
#         if mask_case2.any():
#             mode_city_per_state = (
#                 df[df['city'] != 'Unknown'].groupby('state')['city']
#                 .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else 'Unknown')
#                 .to_dict()
#             )
#             for state, city_mode in mode_city_per_state.items():
#                 mask_fill = mask_case2 & (df['state'] == state)
#                 filled_count = mask_fill.sum()
#                 if filled_count > 0:
#                     df.loc[mask_fill, 'city'] = city_mode
#                     stats["case2_filled"] += filled_count
#                     print(f"[TRACE - STAGE 5] Filled {filled_count} row(s) → missing city for state='{state}' → city='{city_mode}'")
        
#         if stats["case2_filled"] > 0:
#             msg = "Location - Missing City (State Known):\n"
#             msg += f"  • {stats['case2_filled']} record(s) filled with representative city for their state\n"
#             msg += "Why safe: Used real cities from your actual customer base, ensuring realistic segmentation"
#             messages.append(msg)

#         # Case 3: both missing → fill with most frequent pair
#         print("\n[LOG - STAGE 5] Case 3: Filling missing city and state...")
#         mask_case3 = (df['city'] == 'Unknown') & (df['state'] == 'Unknown')
#         if mask_case3.any():
#             valid_pairs = df[(df['city'] != 'Unknown') & (df['state'] != 'Unknown')]
#             if not valid_pairs.empty:
#                 city_mode, state_mode = valid_pairs.groupby(['city', 'state']).size().idxmax()
#                 filled_count = mask_case3.sum()
#                 df.loc[mask_case3, ['city', 'state']] = [city_mode, state_mode]
#                 stats["case3_filled"] = filled_count
#                 print(f"[TRACE - STAGE 5] Filled {filled_count} row(s) → missing city/state → City='{city_mode}', State='{state_mode}'")
#             else:
#                 print("[WARN - STAGE 5] No valid city/state pairs to fill missing both values")
        
#         if stats["case3_filled"] > 0:
#             msg = "Location - Both City & State Missing:\n"
#             msg += f"  • {stats['case3_filled']} record(s) filled with your primary customer location (where most customers are from)\n"
#             msg += "Why safe: Used the most common location in your customer base, minimizing impact on segmentation"
#             messages.append(msg)

#     # Build final message
#     if not messages:
#         final_message = "No missing values found in critical fields (CustomerID, City, State)."
#     else:
#         final_message = "Summary:\n\n" + "\n\n".join(messages)

#     return df, final_message

# # ============================================= (CUSTOMER DATASET) STAGE 6: OUTLIER DETECTION =============================================
# def customer_detect_outliers(df):
#     """Adaptive outlier detection (flag instead of replace)."""
#     print("[LOG - STAGE 6] Running detect_outliers...")
#     message = None
#     if 'age' in df.columns:
#         # Work on a temporary numeric copy to avoid mutating the displayed 'age' values (keep 'Unknown' text)
#         age_num = pd.to_numeric(df['age'], errors='coerce')
#         n = len(df)
#         print(f"[LOG - STAGE 6] Dataset has {n} rows")

#         # Initialize flag column
#         df['is_age_outlier'] = False
#         message = ""

#         if n < 500:
#             # IQR method
#             Q1 = age_num.quantile(0.25)
#             Q3 = age_num.quantile(0.75)
#             IQR = Q3 - Q1
#             lower_bound = Q1 - 1.5 * IQR
#             upper_bound = Q3 + 1.5 * IQR

#             # Flag outliers
#             outlier_mask = (age_num < lower_bound) | (age_num > upper_bound)
#             df.loc[outlier_mask, 'is_age_outlier'] = True
#             outlier_count = outlier_mask.sum()

#             print(f"[LOG - STAGE 6] IQR Applied for {n} rows. Range: [{lower_bound:.1f}, {upper_bound:.1f}] "
#                   f"Outliers flagged: {outlier_count}")
            
#             if outlier_count > 0:
#                 message = (
#                     f"Unusual Age Values Detected:\n\n"
#                     f"We found {outlier_count} customer(s) with ages that seem unusual (either very young or very old) compared to the rest of your data.\n\n"
#                     f"Normal age range in your data: {lower_bound:.0f} to {upper_bound:.0f} years old\n\n"
#                     f"What we did: These customers are marked with a special flag but kept in your dataset. They might be:\n"
#                     f"   - Real customers (e.g., teenagers or senior citizens)\n"
#                     f"   - Data entry mistakes\n\n"
#                     f"You can review them later if needed."
#                 )
#             else:
#                 message = (
#                     f"Age Data Looks Good:\n\n"
#                     f"All customer ages appear normal and consistent. No unusual values detected.\n\n"
#                     f"Typical age range: {lower_bound:.0f} to {upper_bound:.0f} years old"
#                 )

#         else:
#             # Percentile method
#             lower_bound = age_num.quantile(0.01)
#             upper_bound = age_num.quantile(0.99)

#             # Flag outliers instead of capping
#             outlier_mask = (age_num < lower_bound) | (age_num > upper_bound)
#             df.loc[outlier_mask, 'is_age_outlier'] = True
#             outlier_count = outlier_mask.sum()

#             print(f"[LOG - STAGE 6] Percentile method applied for {n} rows. Range: [{lower_bound:.1f}, {upper_bound:.1f}] "
#                   f"Outliers flagged: {outlier_count}")
            
#             if outlier_count > 0:
#                 message = (
#                     f"Unusual Age Values Detected:\n\n"
#                     f"We found {outlier_count} customer(s) with ages that seem unusual (either very young or very old) compared to the rest of your data.\n\n"
#                     f"Normal age range in your data: {lower_bound:.0f} to {upper_bound:.0f} years old\n\n"
#                     f"What we did: These customers are marked with a special flag but kept in your dataset. They might be:\n"
#                     f"   - Real customers (e.g., teenagers or senior citizens)\n"
#                     f"   - Data entry mistakes\n\n"
#                     f"You can review them later if needed."
#                 )
#             else:
#                 message = (
#                     f"Age Data Looks Good:\n\n"
#                     f"All customer ages appear normal and consistent. No unusual values detected.\n\n"
#                     f"Typical age range: {lower_bound:.0f} to {upper_bound:.0f} years old"
#                 )

#     else:
#         print("[LOG - STAGE 6] 'age' column missing, skipping outlier detection")
#         message = "Age column not found. Outlier detection skipped."

#     return df, message

# # ============================================= (CUSTOMER DATASET) DATASET CLEANING PIPELINE =============================================
# def clean_customer_dataset(df, cleaned_output_path):
#     """
#     Main cleaning pipeline for customer dataset.
#     Executes all stages in proper order:
#     0. Column Normalization
#     1. Schema & Column Validation
#     2. Duplicate Entry Removal
#     3. Standardization & Normalization
#     4. Missing Value Handling
#     5. Outlier Detection
#     6. Deduplication
#     Finally, saves the cleaned dataset to the specified path and returns it.
#     """
#     print("🚀 Starting customer data cleaning pipeline...\n", flush=True)
#     messages = [] 
#     report = {"summary": {}, "detailed_messages": {}}
    
#     # Capture initial row count before cleaning
#     initial_row_count = len(df)
#     report["summary"]["initial_rows"] = initial_row_count
    
#     # =======================================================
#     # STAGE 0: NORMALIZE COLUMN NAMES (FROM GENERIC FUNCTION)
#     # =======================================================
#     print("=============== [STAGE 0 START] Normalize Column Names ===============")
#     df = normalize_columns_name(df)
#     print("✅ [STAGE 0 COMPLETE] Column names normalized.\n")

#     # =============================================
#     # STAGE 1: SCHEMA & COLUMN VALIDATION
#     # =============================================
#     print("=============== [STAGE 1 START] Schema & Column Validation ===============")
#     df, optional_msg = customer_check_optional_columns(df)
#     messages.append(optional_msg)
#     report["detailed_messages"]["customer_check_optional_columns"] = optional_msg
    
#     # (FROM GENERIC FUNCTION)
#     customer_mandatory = ["customerid", "city", "state"]
#     df, mandatory_msg = check_mandatory_columns(df,dataset_type="customer", mandatory_columns=customer_mandatory)
#     messages.append(mandatory_msg)
#     report["detailed_messages"]["check_mandatory_columns"] = mandatory_msg
    
#     print(optional_msg)
#     print(mandatory_msg)
#     print("✅ [STAGE 1 COMPLETE] Schema validation done.\n")

#     # ============================================================
#     # STAGE 2: REMOVE DUPLICATE ENTRY ROWS (FROM GENERIC FUNCTION)
#     # ============================================================
#     print("========== [STAGE 2 START] Remove Duplicate Entry Rows ==========")
#     initial_rows = len(df)
#     df, message = remove_duplicate_entries(df)
#     messages.append(message)
#     report["detailed_messages"]["remove_duplicate_entries"] = message
#     report["summary"]["duplicates_removed_rows"] = initial_rows - len(df)
#     print("✅ [STAGE 2 COMPLETE] Duplicate entries removed.\n")

#     # =============================================
#     # STAGE 3: DEDUPLICATION
#     # =============================================
#     print("========== [STAGE 3 START] Deduplication ==========")
#     df, message = deduplicate_customers(df)
#     messages.append(message)
#     report["detailed_messages"]["deduplicate_customers"] = message
#     report["summary"]["rows_after_deduplication"] = len(df)
#     print("✅ [STAGE 3 COMPLETE] Duplicate CustomerIDs deduplicated.\n")

#     # =============================================
#     # STAGE 4: STANDARDIZATION & NORMALIZATION
#     # =============================================
#     print("========== [STAGE 4 START] Standardization & Normalization ==========")
#     df = standardize_customer_id(df)
#     df, dob_msg = standardize_dob(df)
#     messages.append(dob_msg)
#     report["detailed_messages"]["standardize_dob"] = dob_msg
#     df = derive_age_features(df)
#     df = derive_age_group(df)
#     df = drop_dob_after_age_derived(df)
#     df = standardize_gender(df)
#     df = standardize_location(df)
#     print("✅ [STAGE 4 COMPLETE] Standardization and normalization finished.\n")
    
#     # =============================================
#     # STAGE 5: MISSING VALUE HANDLING
#     # =============================================
#     print("========== [STAGE 5 START] Missing Value Handling ==========")
#     df, missing_value_msg = handle_missing_values_customer(df)
#     messages.append(missing_value_msg)
#     report["detailed_messages"]["handle_missing_values_customer"] = missing_value_msg
#     print("✅ [STAGE 5 COMPLETE] Missing values handled.\n")

#     # =============================================
#     # STAGE 6: OUTLIER DETECTION
#     # =============================================
#     print("========== [STAGE 6 START] Outlier Detection ==========")
#     df, outlier_msg = customer_detect_outliers(df)
#     messages.append(outlier_msg)
#     report["detailed_messages"]["customer_detect_outliers"] = outlier_msg
#     print("✅ [STAGE 6 COMPLETE] Outliers handled.\n")

#     # final profiling summary
#     report["summary"].update({
#         "total_rows_final": len(df),
#         "total_columns_final": len(df.columns),
#         "final_columns": list(df.columns),  # Add list of remaining column names
#     })
#     # =============================================
#     # SAVE CLEANED DATASET
#     # =============================================
#     print("========== [FINAL STAGE START] Save Cleaned Dataset ==========")
#     df.to_csv(cleaned_output_path, index=False)
#     print(f"✅ [FINAL STAGE COMPLETE] Cleaned dataset saved at: {cleaned_output_path}\n")
#     print("==========================================================")
#     print("🎉 Data cleaning pipeline completed successfully!\n")
    
#     return df, report # the messages can modify later if needed (now only handle neccessary part), but no return for now, we try return report first

In [ ]:
# # Keep a copy for comparison
# original_customer_df = customer_df.copy()

# # >>>>>>>>>>>>>>>>>>>> (CUSTOMER DATASET) IMPLEMENT CLEANING PIPELINE >>>>>>>>>>>>>>>>>>>
# cleaned_customer_df, report = clean_customer_dataset(customer_df, original_customer_dataset_name)

# # AFTER CLEANING : CLEANED VERSION TO BE STORED BACK TO DATABASE
# cleaned_customer_df.head()

## Perform Data Cleaning Pipeline - OrderDataset

In [ ]:
# # Step 2: Load dataset
# # Replace 'customer_dataset.csv' with your actual file name or path
# order_file_path = "C:/Users/user/OneDrive/Desktop/Onedrive_YuyanDipsy/OneDrive/UM Y4S1/WIA3002 FYP 1 & 2/FYP2/Data/Soapan Santun - Use/2021 - 2025 Order.csv"
# # customer_file_path = "C:/Users/user/OneDrive/Desktop/Onedrive_YuyanDipsy/OneDrive/UM Y4S1/WIA3002 FYP 1 & 2/FYP2/Data/Test Data/customer_dataset_balanced_5033_rows.csv"
# # customer_file_path = "C:/Users/user/OneDrive/Desktop/Onedrive_YuyanDipsy/OneDrive/UM Y4S1/WIA3002 FYP 1 & 2/FYP2/Data/Test Data/customer_dataset_light_7719_rows.csv"
# # customer_file_path = "C:/Users/user/OneDrive/Desktop/Onedrive_YuyanDipsy/OneDrive/UM Y4S1/WIA3002 FYP 1 & 2/FYP2/Data/Test Data/customer_dataset_stress_6047_rows.csv"

# original_order_dataset_name = "2021 - 2025 Order.csv"

# # Read dataset
# order_df = pd.read_csv(order_file_path)

# # Show first few rows (original raw data)
# order_df.head()

In [ ]:
# # Step 1: Import libraries
# # from common_utils import normalize_columns_name, check_mandatory_columns, remove_duplicate_entries, standardize_customer_id;
# import pandas as pd
# import numpy as np
# from datetime import datetime, date
# from fuzzywuzzy import process, fuzz

# # ============================================= (ORDER DATASET) STAGE 0: NORMALIZE COLUMN NAMES =============================================
# # From Generic function: normalize_columns_name

# # ============================================= (ORDER DATASET) STAGE 1: SCHEMA & COLUMN VALIDATION =============================================
# # From Generic function

# # ============================================= (ORDER DATASET) STAGE 2: REMOVE DUPLICATE ENTRY ROW =================================================
# # From Generic function: remove_duplicate_entries

# # ============================================= (ORDER DATASET) STAGE 3: STANDARDIZATION & NORMALIZATION =============================================
# # From Generic function: standardize_customer_id

# def standardized_order_id(df):
#     """Standardize OrderID format (null is '')"""
#     print("[LOG - STAGE 3] Running standardize_order_id...")
#     if 'orderid' in df.columns:
#         # Fill '' with empty string before converting to string
#         df.loc[:, 'orderid'] = df['orderid'].astype(str).str.strip().str.upper()
        
#         # Convert empty string back to NaN
#         df.loc[df['orderid'] == '', 'orderid'] = np.nan
        
#         print("[LOG - STAGE 3] OrderID column standardized (empty -> NaN)")
#     else:
#         print("[LOG - STAGE 3] OrderID column not found, skipping")
#     return df

# def standardize_purchase_item(df):
#     """"Standardize Purchase Item names (NaN preserved)"""
#     print("[LOG - STAGE 3] Running standardized_purchase_item...")
#     if "purchase item" in df.columns:
#         mask = df["purchase item"].notna()
#         df.loc[mask, "purchase item"] = (
#             df.loc[mask, "purchase item"]
#             .astype(str)
#             .str.strip()
#             .str.title()
#         )
#         df.loc[df["purchase item"].str.strip() == "", "purchase item"] = np.nan
#         print("[LOG - STAGE 3] Purchase Item standardized, NaN preserved")
#     else:
#         print("[LOG - STAGE 3] 'purchase item' column not found, skipping")
#     return df

# def standardize_purchase_date(df):
#     """Standardize Purchase Date into separate date and time columns(NaT preserved)"""
#     print("[LOG - STAGE 3] Running standardize_purchase_date...")
#     message = None
#     if "purchase date" in df.columns:
#         # Ensure df is a deep copy (prevents SettingWithCopyWarning)
#         df = df.copy()
        
#         # Clean values
#         df.loc[df["purchase date"].notna(), "purchase date"] = df.loc[df["purchase date"].notna(), "purchase date"].astype(str).str.strip()

#         # Convert to datetime
#         df.loc[:, "purchase datetime"] = pd.to_datetime(
#             df["purchase date"], errors="coerce", dayfirst=True
#         )

#         # Detect which rows have time info
#         has_time_mask = df["purchase date"].notna() & df["purchase date"].str.contains(":", regex=False)
        
#         # Create standardized columns
#         df.loc[:, "purchase date"] = df["purchase datetime"].dt.strftime("%Y-%m-%d")
#         df.loc[:, "purchase date"] = df["purchase date"].replace("NaT", pd.NA)
#         df.loc[:, "purchase time"] = None
#         df.loc[has_time_mask, "purchase time"] = (
#             df.loc[has_time_mask, "purchase datetime"].dt.strftime("%H:%M:%S")
#         )
        
#         # Drop intermediate column
#         df.drop(columns=["purchase datetime"], inplace=True, errors="ignore")
        
#         if has_time_mask.any():
#             message = (
#                 "Since your Purchase Date information have include time information, "
#                 "so a separate 'purchase time' column has been derived to be used for segmentation later."
#             )
#     else:
#         print("[WARN - STAGE 3] 'purchase date' column not found, skipping.")
#     print("[LOG - STAGE 3] Purchase date standardization complete, NaN preserved.")
#     return df, message

# def standardized_item_price_and_total_spend(df):
#     """
#     Standardize 'item price' and 'total spend' columns in the dataset.

#     Steps performed for each column:
#     1. Remove all non-numeric characters except digits, decimal points, and negative signs
#     (e.g., currency symbols like RM, $, MYR are removed)
#     2. Convert the resulting strings to numeric values; invalid or non-convertible entries are set to NaN
#     3. Round numeric values to 2 decimal places

#     This ensures the columns are clean, numeric, and ready for analysis.
#     """

#     print("[LOG - STAGE 3] Running standardized_item_price_and_total_spend...")

#     for col in ["item price", "total spend"]:
#         if col in df.columns:
#             # Step 1: Remove currency symbols and text
#             df[col] = (
#                 df[col]
#                 .astype(str)
#                 .str.replace(r"[^\d\.\-]", "", regex=True)
#             )

#             # Step 2: Convert to numeric, coercing invalid values to NaN
#             df[col] = pd.to_numeric(df[col], errors="coerce")

#             # Step 3: Round to 2 decimal places
#             df[col] = df[col].round(2)

#             print(f"[LOG - STAGE 3] {col} standardized: numeric, 2 decimal places, NaN preserved")
#         else:
#             print(f"[LOG - STAGE 3] '{col}' column not found, skipping")

#     return df

# def standardize_purchase_quantity(df):
#     """Standardize Purchase Quantity to integer (NaN preserved)"""
#     print("[LOG - STAGE 3] Running standardize_purchase_quantity...")

#     if "purchase quantity" in df.columns:
#         # Remove non-numeric characters (like pcs, x, units, etc.)
#         df["purchase quantity"] = (
#             df["purchase quantity"]
#             .astype(str)
#             .str.replace(r"[^\d\.\-]", "", regex=True)  # keep digits only
#         )

#         # Convert to numeric (NaN for invalid)
#         df["purchase quantity"] = pd.to_numeric(df["purchase quantity"], errors="coerce")

#         # Round any decimals (e.g. 2.5 → 2)
#         df["purchase quantity"] = df["purchase quantity"].round(0).astype("Int64")

#         print("[LOG - STAGE 3] Purchase quantity standardized to integer format, NaN preserved")
#     else:
#         print("[LOG - STAGE 3] 'purchase quantity' column not found, skipping")

#     return df

# def standardize_transaction_method(df):
#     """
#     Standardize 'transaction method' into categories:
#     ['Cash', 'Card', 'E-Wallet', 'Online Banking', 'Auto-Debit', 'Cheque']
#     (Unknown represents missing values)
#     """
#     import re

#     print("[LOG - STAGE 3] Running standardize_transaction_method...")

#     if "transaction method" not in df.columns:
#         print("[LOG - STAGE 3] 'transaction method' column not found, skipping.")
#         return df
    
#     # Step1: Only process non-null values
#     mask_notna = df["transaction method"].notna()
#     df.loc[mask_notna, "transaction method"] = (
#         df.loc[mask_notna, "transaction method"].astype(str).str.lower().str.strip()
#     )
    
#     # Step 2: Define patterns (non-capturing groups)
#     patterns = {
#         "Cash": r"\b(?:cash|tunai|otc|counter)\b",
#         "Card": r"\b(?:card|visa|master|credit|debit|amex|credit.?debit)\b",
#         "E-Wallet": r"\b(?:tng|touch\s*n\s*go|grab\s*pay|grabpay|boost|shopee\s*pay|shopeepay|spaylater|duitnow|ewallet|e-?wallet|qr|qr\s*pay|qrcode)\b",
#         "Online Banking": r"\b(?:bank|transfer|fpx|online\s*payment|maybank2u|cimbclicks|duitnow\s*qr|public\s*bank)\b",
#         "Auto-Debit": r"\b(?:auto.?debit|standing|recurring|subscription|auto\s*pay)\b",
#         "Cheque": r"\b(?:cheque|cek|check)\b",
#     }

#     # Step 4: Apply vectorized regex matching
#     for category, pattern in patterns.items():
#         mask = df["transaction method"].str.contains(pattern, flags=re.IGNORECASE, na=False, regex=True)
#         df.loc[mask, "transaction method"] = category

#     mask_unmatched = mask_notna & ~df["transaction method"].isin(patterns.keys())
#     df.loc[mask_unmatched, "transaction method"] = pd.NA

#     print("[LOG - STAGE 3] Transaction method standardized successfully, NaN preserved.")
#     return df

# # ============================================= (ORDER DATASET) STAGE 4: MISSING VALUE HANDLING =============================================
# def handle_missing_values_order(df):
#     """
#     Strategy (for SME context):
#     - Drop rows if (orderid, customerid, purchase date) are missing
#     - Drop rows if (purchase time) is missing (when time column exists)
#     - Drop rows if both (item price and total spend) are missing
#     - Drop rows if (item price) is missing (critical for calculations)
#     - Fill or calculate non-critical missing fields logically:
#         - purchase quantity: calculate from (total spend / item price) if possible, otherwise fill with 1
#         - total spend: calculate from (item price × quantity) if possible, otherwise drop row
#         - transaction method: fill with "Unknown"
    
#     Note: purchase item is NOT filled because product name is important for business analysis
#     """

#     initial_count = len(df)
#     messages = []
#     stats = {
#         "critical_ids_removed": 0,
#         "purchase_time_removed": 0,
#         "no_financial_removed": 0,
#         "item_price_removed": 0,
#         "total_spend_removed": 0,
#         "qty_calculated": 0,
#         "total_spend_calculated": 0,
#         "transaction_method_filled": 0
#     }

#     # Drop rows missing critical identifiers
#     critical_cols = ["orderid", "customerid", "purchase date"]
#     existing_critical = [c for c in critical_cols if c in df.columns]
#     before_critical = len(df)
#     df = df.dropna(subset=existing_critical)
#     stats["critical_ids_removed"] = before_critical - len(df)
#     print(f"[LOG - STAGE 4] Dropped rows with missing critical identifiers: {stats['critical_ids_removed']}")
    
#     if "purchase time" in df.columns:
#         # Drop rows where purchase time is null
#         before_drop = len(df)
#         df = df.dropna(subset=["purchase time"])
#         stats["purchase_time_removed"] = before_drop - len(df)
#         print(f"[LOG - STAGE 4] Dropped {stats['purchase_time_removed']} rows with missing purchase time")

#     # Drop rows missing both financial info
#     before_financial = len(df)
#     df = df.dropna(subset=["item price", "total spend"], how="all")
#     stats["no_financial_removed"] = before_financial - len(df)
#     print(f"[LOG - STAGE 4] Dropped {stats['no_financial_removed']} rows with no financial info")

#     for col in ["item price", "total spend"]:
#         if col in df.columns:
#             df[col] = df[col].astype(str).str.replace(r"[^\d\.\-]", "", regex=True)
#             df[col] = pd.to_numeric(df[col], errors="coerce").round(2)
            
#             # Remove zero or negative values that are logically impossible
#             df.loc[df[col] <= 0, col] = np.nan
#             zero_count = df[col].isna().sum()
#             if zero_count > 0:
#                 print(f"[WARN - STAGE 4] {zero_count} rows in '{col}' are zero or invalid and set to NaN")

#     # Handle purchase quantity: calculate if possible
#     if "purchase quantity" in df.columns:
#         mask_missing_qty = df['purchase quantity'].isna() & df['total spend'].notna() & df['item price'].notna()
#         calculated_count = mask_missing_qty.sum()
#         df.loc[mask_missing_qty, 'purchase quantity'] = (df.loc[mask_missing_qty, 'total spend'] / df.loc[mask_missing_qty, 'item price']).round(0).astype('Int64')
#         stats["qty_calculated"] = calculated_count
#         # Remaining missing → 1 as fallback
#         df['purchase quantity'] = df['purchase quantity'].fillna(1)

#     # Handle item price - but very rare happened so I decided to simply drop that row
#     if "item price" in df.columns:
#         before_drop = len(df)
#         df = df.dropna(subset=["item price"])
#         stats["item_price_removed"] = before_drop - len(df)
#         print(f"[LOG - STAGE 4] Dropped {stats['item_price_removed']} rows with missing 'item price' (critical for calculations)")

#     # Handle total spend
#     if {"item price", "purchase quantity", "total spend"}.issubset(df.columns):
#         mask_missing_total = df["total spend"].isna() & df["item price"].notna() & df["purchase quantity"].notna()
#         calculated_total_count = mask_missing_total.sum()
#         df.loc[mask_missing_total, "total spend"] = (
#             df.loc[mask_missing_total, "item price"] * df.loc[mask_missing_total, "purchase quantity"]
#         ).round(2)
#         stats["total_spend_calculated"] = calculated_total_count
#         # Any remaining missing → drop (very rare)
#         before_drop = len(df)
#         df = df.dropna(subset=["total spend"])
#         stats["total_spend_removed"] = before_drop - len(df)
#         print(f"[LOG - STAGE 4] Dropped {stats['total_spend_removed']} rows with missing 'total spend' after calculation")

#     # Transaction method → Unknown
#     if "transaction method" in df.columns:
#         before_fill = df["transaction method"].isna().sum()
#         df["transaction method"] = df["transaction method"].replace(["", "NaN", None], np.nan)
#         df["transaction method"] = df["transaction method"].fillna("Unknown")
#         stats["transaction_method_filled"] = before_fill

#     # Build message
#     dropped_total = initial_count - len(df)
    
#     if dropped_total > 0:
#         messages.append(f"{dropped_total} order(s) removed due to missing critical information:")
#         if stats["critical_ids_removed"] > 0:
#             messages.append(f"  - {stats['critical_ids_removed']} order(s) without OrderID, CustomerID, or Purchase Date")
#         if stats["purchase_time_removed"] > 0:
#             messages.append(f"  - {stats['purchase_time_removed']} order(s) without Purchase Time")
#         if stats["no_financial_removed"] > 0:
#             messages.append(f"  - {stats['no_financial_removed']} order(s) without any price or total spend information")
#         if stats["item_price_removed"] > 0:
#             messages.append(f"  - {stats['item_price_removed']} order(s) without Item Price (needed for calculations)")
#         if stats["total_spend_removed"] > 0:
#             messages.append(f"  - {stats['total_spend_removed']} order(s) without Total Spend even after calculation")
    
#     calculation_msgs = []
#     if stats["qty_calculated"] > 0:
#         calculation_msgs.append(f"  - Calculated Purchase Quantity for {stats['qty_calculated']} order(s) using Total Spend / Item Price")
#     if stats["total_spend_calculated"] > 0:
#         calculation_msgs.append(f"  - Calculated Total Spend for {stats['total_spend_calculated']} order(s) using Item Price x Quantity")
#     if stats["transaction_method_filled"] > 0:
#         calculation_msgs.append(f"  - Filled {stats['transaction_method_filled']} missing Transaction Method(s) with 'Unknown'")
    
#     if calculation_msgs:
#         messages.append("\nData Filled/Calculated:")
#         messages.extend(calculation_msgs)
    
#     if not messages:
#         final_message = "All order records have complete information. No missing values found."
#     else:
#         final_message = "Missing Value Handling Summary:\n\n" + "\n".join(messages)
    
#     # Final summary
#     print(f"[LOG - STAGE 4] Dropped total {dropped_total} rows ({dropped_total/initial_count:.2%}) due to missing critical data")
#     print(f"[LOG - STAGE 4] Dataset now has {len(df)} rows after missing value handling")

#     return df, final_message

# # ============================================= (ORDER DATASET) STAGE 5: OUTLIER DETECTION =============================================
# def order_detect_outliers(df):
#     """
#     Stage 5: Detect and flag outliers for order dataset (NO capping/replacement).
#     - Apply IQR method if dataset < 500 rows.
#     - Apply percentile method (1st–99th) if dataset >= 500 rows.
#     - Columns: purchase quantity, total spend.
#     - Outliers are FLAGGED only, original values preserved.
#     """
#     print(f"[LOG - STAGE 5] Dataset has {len(df)} rows")

#     numeric_cols = ["purchase quantity", "total spend"]
#     df = df.copy()
    
#     messages = []
#     outlier_info = []
#     dataset_size = len(df)
    
#     # Initialize flag columns
#     df['is_quantity_outlier'] = False
#     df['is_spend_outlier'] = False
    
#     for col in numeric_cols:
#         if col not in df.columns:
#             continue

#         if df[col].dropna().empty:
#             print(f"[WARN - STAGE 5] {col} is empty or missing, skipping.")
#             continue

#         if len(df) < 500:
#             # IQR Method
#             Q1 = df[col].quantile(0.25)
#             Q3 = df[col].quantile(0.75)
#             IQR = Q3 - Q1
#             lower = Q1 - 1.5 * IQR
#             upper = Q3 + 1.5 * IQR
#             method = "IQR method"
#         else:
#             # Percentile Method
#             lower = df[col].quantile(0.01)
#             upper = df[col].quantile(0.99)
#             method = "Percentile method"

#         # Flag outliers (do NOT modify original values)
#         outlier_mask = (df[col] < lower) | (df[col] > upper)
#         outliers_count = outlier_mask.sum()
        
#         # Set appropriate flag column
#         if col == "purchase quantity":
#             df.loc[outlier_mask, 'is_quantity_outlier'] = True
#         elif col == "total spend":
#             df.loc[outlier_mask, 'is_spend_outlier'] = True
        
#         print(f"[LOG - STAGE 5] {method} applied on '{col}'. Range: [{lower:.2f}, {upper:.2f}]. Outliers flagged: {outliers_count}")
        
#         if outliers_count > 0:
#             col_display = col.replace('_', ' ').title()
#             outlier_info.append(f"  - {col_display}: {outliers_count} unusual value(s) detected (may indicate high-value customers)")

#     # Build message
#     if outlier_info:
#         messages.append("Unusual Values Detected:")
#         messages.append("\nWe found some extremely high or low values in your order data.")
#         messages.append("\nWhat we did: Flagged these values for your attention. Original values are preserved.")
#         messages.append("\nDetails:")
#         messages.extend(outlier_info)
#         messages.append("\nNote: These outliers might represent VIP customers or bulk buyers. They will be considered during segmentation to help identify high-value customer groups.")
#     else:
#         messages.append("Order Values Look Good:")
#         messages.append("\nAll order values (quantities and amounts) appear normal and consistent. No unusual values detected.")

#     final_message = "\n".join(messages)
    
#     print("✅ [STAGE 5 COMPLETE] Outliers flagged successfully (values preserved).")
#     return df, final_message

# # ============================================= (ORDER DATASET) DATASET CLEANING PIPELINE =============================================
# def clean_order_dataset(df, cleaned_output_path):
#     print("🚀 Starting order data cleaning pipeline...\n")
#     messages = [] 
#     report = {"summary": {}, "detailed_messages": {}}
    
#     # Capture initial row count before cleaning
#     initial_row_count = len(df)
#     report["summary"]["initial_rows"] = initial_row_count
    
#     # =======================================================
#     # STAGE 0: NORMALIZE COLUMN NAMES (FROM GENERIC FUNCTION)
#     # =======================================================
#     print("========== [STAGE 0 START] Normalize Column Names ==========")
#     df = normalize_columns_name(df)
#     print("✅ [STAGE 0 COMPLETE] Column names normalized.\n")
    
#     # =============================================
#     # STAGE 1: SCHEMA & COLUMN VALIDATION
#     # =============================================
#     print("========== [STAGE 1 START] Schema & Column Validation ==========")
#     order_mandatory = ["orderid", "customerid", "purchase item", "purchase date", "item price", "purchase quantity", "total spend", "transaction method"]
#     df, mandatory_msg = check_mandatory_columns(df, dataset_type="order", mandatory_columns=order_mandatory)
#     messages.append(mandatory_msg)
#     report["detailed_messages"]["check_mandatory_columns"] = mandatory_msg
#     print(mandatory_msg)
#     print("✅ [STAGE 1 COMPLETE] Schema validation done.\n")
    
#     # ============================================================
#     # STAGE 2: REMOVE DUPLICATE ENTRY ROWS (FROM GENERIC FUNCTION)
#     # ============================================================
#     print("========== [STAGE 2 START] Remove Duplicate Entry Rows ==========")
#     initial_rows = len(df)
#     df, message = remove_duplicate_entries(df)
#     messages.append(message)
#     report["detailed_messages"]["remove_duplicate_entries"] = message
#     report["summary"]["duplicates_removed_rows"] = initial_rows - len(df)
#     print("✅ [STAGE 2 COMPLETE] Duplicate entries removed.\n")
    
#     # =============================================
#     # STAGE 3: STANDARDIZATION & NORMALIZATION
#     # =============================================
#     print("========== [STAGE 3 START] Standardization & Normalization ==========")
#     df = standardized_order_id(df)
#     df = standardize_customer_id(df)
#     df = standardize_purchase_item(df)
#     df, standardize_purchaseDateMessage = standardize_purchase_date(df)
#     messages.append(standardize_purchaseDateMessage)
#     report["detailed_messages"]["standardize_purchase_date"] = standardize_purchaseDateMessage
#     df = standardized_item_price_and_total_spend(df)
#     df = standardize_purchase_quantity(df)
#     df = standardize_transaction_method(df)
#     print("✅ [STAGE 3 COMPLETE] Standardization and normalization finished.\n")

#     # ===============================================
#     # STAGE 4: MISSING VALUE HANDLING
#     # ===============================================
#     print("========== [STAGE 4 START] Missing Value Handling ==========")
#     df, missing_value_msg = handle_missing_values_order(df)
#     messages.append(missing_value_msg)
#     report["detailed_messages"]["handle_missing_values_order"] = missing_value_msg
#     print("✅ [STAGE 4 COMPLETE] Missing values handled.\n")
    
#     # =============================================
#     # STAGE 5: OUTLIER DETECTION
#     # =============================================
#     print("========== [STAGE 5 START] Outlier Detection ==========")
#     df, outlier_msg = order_detect_outliers(df)
#     messages.append(outlier_msg)
#     report["detailed_messages"]["order_detect_outliers"] = outlier_msg
#     print("✅ [STAGE 5 COMPLETE] Outliers handled.\n")
    
#     # final profiling summary
#     report["summary"].update({
#         "total_rows_final": len(df),
#         "total_columns_final": len(df.columns),
#         "final_columns": list(df.columns),  # Add list of remaining column names
#     })
#     # =============================================
#     # SAVE CLEANED DATASET
#     # =============================================
#     print("========== [FINAL STAGE START] Save Cleaned Dataset ==========")
#     # base_name, ext = os.path.splitext(original_order_dataset_name)
#     df.to_csv(cleaned_output_path, index=False)
#     print(f"✅ [FINAL STAGE COMPLETE] Cleaned dataset saved at: {cleaned_output_path}\n")

#     print("==========================================================")
#     print("🎉 Data cleaning pipeline completed successfully!\n")
#     return df, report
#     # later add on return clean file name

In [ ]:
# import subprocess
# import os

# # Define absolute paths
# script_path = "C:/Y3S2/FYP/server/python/cleaningPipeline/cleaning_main.py"
# temp_file_path = "C:/Users/user/OneDrive/Desktop/Onedrive_YuyanDipsy/OneDrive/UM Y4S1/WIA3002 FYP 1 & 2/FYP2/Data/Soapan Santun - Use/2021 - 2025 Customer - Copy.csv"
# original_file_name = "2021 - 2025 Customer - Copy.csv"
# dataset_type = "customer"

# # Construct the command with absolute paths
# command = [
#     "python", 
#     script_path,
#     "--type", dataset_type,
#     "--temp_file_path_with_filename", temp_file_path,
#     "--original_file_name", original_file_name
# ]

# # Run the cleaning pipeline
# result = subprocess.run(
#     command, 
#     capture_output=True, 
#     text=True,
#     cwd=os.path.dirname(script_path)  # Set working directory to script's folder
# )

# # Check results
# if result.returncode == 0:
#     print("✅ Cleaning succeeded!")
#     print(result.stdout)
# else:
#     print("❌ Cleaning failed!")
#     print(result.stderr)